### Perspective API Exploration

First, we have a dataset of Wikipedia comments made available by Jigsaw, a subsidiary of Google that created the Perspective tool. The dataset includes a unique comment id, the text of the comment, and a series of binary labels applied by human raters: "toxic," "severe_toxic," "obscene," "threat," "insult," and "identity_hate." I have appended the "score" column, which represents the toxicity score assigned to the comment text by the live version of the Perspective API. The data is available under a CC0 license.

In [3]:
import pandas as pd
import time

df = pd.read_csv('labeled_and_scored_comments.csv')

In [4]:
df.sort_values(['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
1158,61a85266de70cca0,Можешь говорить по русски.,0.000004,0,0,0,0,0,0
5246,030a9ba0b32b4456,Rodzina Krusenstern Potomkowie Admirała Adama ...,0.000009,0,0,0,0,0,0
11040,a6814193ac6b4705,The opening paragraph states that the game is ...,0.000091,0,0,0,0,0,0
24192,f0d6dee96e057a5b,The book is used as a source in a number of ar...,0.000116,0,0,0,0,0,0
2332,cbb9268d3c7429c1,Vals Vienés \n |Rumba \n |Cha Cha Chá(ciclo 7),0.000131,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
19994,1429a2c930688317,you fucking piece of shit,0.997278,1,1,1,0,1,0
28728,d82e45eb103ad07b,fuck off you stupid aspy asshole,0.997736,1,1,1,0,1,0
25535,e9b92c6c6b01aeef,FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIE...,0.997982,1,1,1,0,1,0
25945,e783fd267f3a9d3b,FUCK WIKIPEDIA ON WHEELS! \n\nFuck off wikiped...,0.998136,1,1,1,0,1,0


I've also included a function to make calls to the Perspective API for your own testing. You will need to generate your own API key according to the instructions in the assignment.

In [33]:
from googleapiclient.discovery import build
import json

def get_toxicity_score(comment):
    
  API_KEY = '---' # Put your API key here
    
  client = build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
  )

  analyze_request = {
  'comment': { 'text': comment },
  'requestedAttributes': {'TOXICITY': {}}
  }
    
  response = client.comments().analyze(body=analyze_request).execute()
  toxicity_score = response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]
    
  return toxicity_score

We can call this function with original comments like this:

In [6]:
get_toxicity_score("I love you")

0.040628307

Due to the constraint of one request per second, we should insert a time.sleep() statement between consecutive API calls, which will pause execution for one second.

In [34]:
threshold = 0.58 # After sorting the CSV by score, I started my search for a threshold at 0.50. Seeking to determine a point at which the majority of comments were at least mean-spirited
# and impolite, I worked my way up and landed at 0.58.

comment_list = ['whats up buddy' , 'I cannot believe you thought that was a good edit. do better.' , 'I would like to suggest the addition of a history and accolades section to the team\'s page.'
                , 'you imbecile' , 'Look here kid, if you don\'t stop suggesting troll edits I will have to report you. This is absolutely a threat, and it carries signficant weight. I am not a mod, however, upon checking my user history you will find that I am a respected contributor in the My Little Pony Wiki community, and am responsible for the neutralizing of many trolls. It\'s time to shape up, bucko, or your ass will be outta here.'
   , 'Why do I spend my time on this fucking stupid website with these fucking stupid people?'  , 'yes it does' , 'y cant u accept this common knowledge? every1 knows there was a second shooter @ jfk... except u?'
               , 'I CANT STAND YOU FUCKING AMERICANS AND YOUR FUCKING M/D/Y SYSTEM!!!! IT NEVER FUCKING ENDS DOES IT? YOU AND YOUR NEED TO BE DIFFERENT. CONTRIVED CULTURE FULL OF SOULLESS TWATS!!' , 'wut?' , 'Yeah according to my primary source, your estimation of her birth being in 1857 is incorrect. Mine says 1855, and is a document dated then.' , 'Bro thats fucking awesome. Keep it up.' , 'you dunce' 
               , 'I never called him a bitch. I only asked him to put away his opinion on the topic and consider that there are multiple sides.' , 'No. Respectfully, I don\'t see why the word \'fuck\' should be included in the article' , 'you 5h!the4d']
# The comments in the comment_list do not reflect my values, opinions, or beliefs. They were created with the idea of being an assormtent of internet comments, in different tones, at different levels of toxicity.
score_list = []
toxic_list = [] # I chose the label 'toxic' because it is the most basic determinant of negativity in a comment, and in the given dataset, I found it to be the most
# reliable measure of if a comment was 'good' versus 'bad.' The other labels seemed a bit more specific, and open to interpretation.
# For example, amny of the highest scoring comments did not get a 1 on 'identity hate,' so choosing that label might not work well as many of the most toxic comments above the score threshold in the test set may not contain identity hate.
toxic = 0
for comment in comment_list:
    score = get_toxicity_score(comment)
    score_list.append(score)
    if score >= threshold:
        toxic = 1
        toxic_list.append(toxic)
    else:
        toxic = 0
        toxic_list.append(toxic)
    print(comment, score, toxic)
    time.sleep(1)
    
# Hypothesis: I believe that the Perspective API scores comments based on how often they contain certain words and phrases.
# I think that these words/phrases, even when employed in a clearly not-toxic manner, will bump up a comment's score.
# I think that if the highest scores are achieved by comments with several words, but also with a more frequent use of the score-increasing words.
# I imagine that more commonly used insults are likely to score higher than less common counterparts which mean the same think, when used in the same context.

whats up buddy 0.09924392 0
I cannot believe you thought that was a good edit. do better. 0.15743166 0
I would like to suggest the addition of a history and accolades section to the team's page. 0.012772553 0
you imbecile 0.8546526 1
Look here kid, if you don't stop suggesting troll edits I will have to report you. This is absolutely a threat, and it carries signficant weight. I am not a mod, however, upon checking my user history you will find that I am a respected contributor in the My Little Pony Wiki community, and am responsible for the neutralizing of many trolls. It's time to shape up, bucko, or your ass will be outta here. 0.5958905 1
Why do I spend my time on this fucking stupid website with these fucking stupid people? 0.96751314 1
yes it does 0.019158829 0
y cant u accept this common knowledge? every1 knows there was a second shooter @ jfk... except u? 0.31714454 0
I CANT STAND YOU FUCKING AMERICANS AND YOUR FUCKING M/D/Y SYSTEM!!!! IT NEVER FUCKING ENDS DOES IT? YOU AND YOU

In [8]:
#df.head()

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0.075638,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0.450459,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0.667964,0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0.068434,0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0.151724,0,0,0,0,0,0


In [10]:
#from sklearn.metrics import confusion_matrix

In [27]:
testSet_df = pd.DataFrame({'comment': comment_list, 'score': score_list, 'toxic': toxic_list})

In [30]:
testSet_df.head()

,comment,score,toxic
0,whats up buddy,0.099244,0
1,I cannot believe you thought that was a good e...,0.157432,0
2,I would like to suggest the addition of a hist...,0.012773,0
3,you imbecile,0.854653,1
4,"Look here kid, if you don't stop suggesting tr...",0.595890,1
